### Data Umbrella PyMC Open Source Working Sessions
#### July to August, 2022
#### Author:  Reshama Shaikh

#### Geolocation Data
-  
Want to get geo codes for participants for sprints

#### Notes
- Input data: `'../data/2022_07_du_pymc.csv'`
- Output data: 
- Output data: `'../data/derived_geocodes_pymc.csv'`

---

## Date

In [1]:
from datetime import date

current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-08-23


## Import libraries


In [2]:
import pandas as pd 
import numpy as np
import requests
import time
import csv
from pprint import pprint

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
%load_ext watermark
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.25.0

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Git hash: 26b6442cf855859c1f79aa9548a9a5a4bdcf0d63

numpy   : 1.20.2
requests: 2.26.0
pandas  : 1.3.3
csv     : 1.0
plotly  : 5.4.0



## Set pandas view options

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read in csv file

In [6]:
path_datain = "../data/data_raw/"


In [7]:
df_use = pd.read_csv(path_datain+'2022_07_du_pymc.csv')

In [8]:
df_use['location'] = df_use.apply(lambda x:'%s %s' % (x['city'],x['country']),axis=1)

In [9]:
df_use.shape

(80, 32)

In [10]:
df_use.head(2)

,Timestamp,city,state_province,github,country,gender,pronouns,pre_oh,session_1,session_2,session_3,post_oh,status,submitted_pr,joined_discord,role,urp,sessions,learn_of_sprint,prior_sprints,os_experience,prior_os_pr,git,prog_langs,python_experience,used_pymc,pymc_repo_familiar,why_in_sprint,prep_work,primary_spoken_language,language_translation,location
0,6/13/2022 18:54:55,Rochester,New York,https://github.com/majamil16,USA,Woman,she/her,NaN,NaN,NaN,NaN,NaN,0.0,0,1.0,"Data Scientist, Engineer, Software developer","Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,Intermediate,Yes,"comfortable submitting a PR, comfortable setti...","Python, C+ / C++, Java",Advanced,I am not at all familiar with PyMC.,"I am familiar, I have looked at the codebase.","Looking to grow my skills as a developer, and ...",NaN,English,No,Rochester USA
1,6/13/2022 19:56:59,New York,New York,https://github.com/mariyayb,USA,Woman,she/her,1.0,1,NaN,NaN,NaN,2.0,1,1.0,Data Scientist,I prefer not to answer this question.,"Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,No experience,No,comfortable setting up virtual environment,Python,Intermediate,I am not at all familiar with PyMC.,I am not at all familiar with this library.,gain experience and learn,Yes,English,No,New York USA


---

In [11]:
from opencage.geocoder import OpenCageGeocode

https://amaral.northwestern.edu/blog/getting-long-lat-list-cities
#!pip install opencage

In [12]:
# log in using GH authentication
# https://opencagedata.com/dashboard#geocoding

key = "aa82453ae4e44055a43dd954ce8c4bef"  # get api key from:  https://opencagedata.com
key = "2af537650a554a2b80705c9aeb45f8a6"
geocoder = OpenCageGeocode(key)


In [13]:
loc_codes =[]

def get_code(place, logprint):
    query=place
    results = geocoder.geocode(query)
    #pprint (results)

    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']

    continent = results[0]['components']['continent']
    country = results[0]['components']['country']
    #state = results[0]['components']['state']
    #city = results[0]['components']['city']
    iso_alpha = results[0]['annotations']['currency']['iso_code']
    iso_num = results[0]['annotations']['currency']['iso_numeric']
    
    #temp = [place, continent, country,  city, lat, lng]
    temp = [place, continent, country, iso_alpha, iso_num,  lat, lng]

    loc_codes.append(temp)
    if logprint==1:
        print('-'*10)
        pprint (results)
        print('-'*10)
        print(place)
        print(continent)
        print(country)
        #print(state)
        #print(city)
        print(lat,lng)
        print('-'*10)
        #print(iso_alpha, iso_num)
    print(temp)
    return temp


In [14]:
df_loc=df_use.copy()

count=-1
loc_list = list()

for i, row in df_loc.iterrows():
    max_rows = 300
    #max_rows = 10
    if count <= max_rows:
        count+=1
        #print(count)
        #print(f"Index: {i}", f"{row['location']}", f"{row['location2']}")
        temp = row['location']
        #print(temp)
        temp2 = get_code(temp, 0)
        #print(temp2)
        loc_list.append(temp2)
        #print(f"{row['city']}")
        
    if count % 10 ==0:
        print(count)

['Rochester USA', 'North America', 'United States', 'USD', '840', 43.157285, -77.615214]
0
['New York USA', 'North America', 'United States', 'USD', '840', 40.7127281, -74.0060152]
['Winneba Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.35006, -0.62134]
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Pune India', 'Asia', 'India', 'INR', '356', 18.521428, 73.8544541]
['São Paulo Brasil', 'South America', 'Brazil', 'BRL', '986', -23.5506507, -46.6333824]
['São Paulo Brasil', 'South America', 'Brazil', 'BRL', '986', -23.5506507, -46.6333824]
10
['Accra Ghana', 'Africa', 'Ghana', 'GHS', '936', 5.5571096, -0.2012376]
['Berlin Germany', 'Europe', 'Germany', 'EUR', '978', 52.5170365, 13.

In [15]:
type(loc_list)

list

In [16]:
import csv

fields = ['location', 'continent_o', 'country_o', 'iso_alpha', 'iso_num',  'lat', 'lng']

with open('../data/data_derived/derived_geocodes_pymc.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(loc_list)

In [17]:
len(loc_list)

80

In [18]:
#print("\nloc_list")
#pprint(loc_list[0:3]) 

In [19]:
from pandas import DataFrame

df_locinfo = DataFrame(loc_list, columns=['location', 'continent_o', 'country_o', \
                                          'iso_alpha', 'iso_num',  'lat', 'lng'])

In [20]:
len(df_locinfo)

80

In [21]:
df_locinfo2 = df_locinfo.drop_duplicates(subset=['location'])

In [22]:
len(df_locinfo2)

53

In [23]:
df_locinfo.head(3)

,location,continent_o,country_o,iso_alpha,iso_num,lat,lng
0,Rochester USA,North America,United States,USD,840,43.157285,-77.615214
1,New York USA,North America,United States,USD,840,40.712728,-74.006015
2,Winneba Ghana,Africa,Ghana,GHS,936,5.350060,-0.621340


In [24]:
df_use.head(3)

,Timestamp,city,state_province,github,country,gender,pronouns,pre_oh,session_1,session_2,session_3,post_oh,status,submitted_pr,joined_discord,role,urp,sessions,learn_of_sprint,prior_sprints,os_experience,prior_os_pr,git,prog_langs,python_experience,used_pymc,pymc_repo_familiar,why_in_sprint,prep_work,primary_spoken_language,language_translation,location
0,6/13/2022 18:54:55,Rochester,New York,https://github.com/majamil16,USA,Woman,she/her,NaN,NaN,NaN,NaN,NaN,0.0,0,1.0,"Data Scientist, Engineer, Software developer","Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,Intermediate,Yes,"comfortable submitting a PR, comfortable setti...","Python, C+ / C++, Java",Advanced,I am not at all familiar with PyMC.,"I am familiar, I have looked at the codebase.","Looking to grow my skills as a developer, and ...",NaN,English,No,Rochester USA
1,6/13/2022 19:56:59,New York,New York,https://github.com/mariyayb,USA,Woman,she/her,1.0,1,NaN,NaN,NaN,2.0,1,1.0,Data Scientist,I prefer not to answer this question.,"Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,No experience,No,comfortable setting up virtual environment,Python,Intermediate,I am not at all familiar with PyMC.,I am not at all familiar with this library.,gain experience and learn,Yes,English,No,New York USA
2,6/14/2022 12:13:11,Winneba,Central Region,https://github.com/elijah-appiah,Ghana,Man,he/him,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,Student,"Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,Beginner,No,"used Git, but am not comfortable","Python, R",Intermediate,I am not at all familiar with PyMC.,I am not at all familiar with this library.,I aspire to be a Data Scientist and a Python D...,NaN,English,No,Winneba Ghana


In [25]:
df_locinfo2 = pd.merge(df_locinfo2, df_use, on=['location'], how='left')

In [26]:
len(df_locinfo2)

80

In [27]:
df_locinfo2

,location,continent_o,country_o,iso_alpha,iso_num,lat,lng,Timestamp,city,state_province,github,country,gender,pronouns,pre_oh,session_1,session_2,session_3,post_oh,status,submitted_pr,joined_discord,role,urp,sessions,learn_of_sprint,prior_sprints,os_experience,prior_os_pr,git,prog_langs,python_experience,used_pymc,pymc_repo_familiar,why_in_sprint,prep_work,primary_spoken_language,language_translation
0,Rochester USA,North America,United States,USD,840,43.157285,-77.615214,6/13/2022 18:54:55,Rochester,New York,https://github.com/majamil16,USA,Woman,she/her,NaN,NaN,NaN,NaN,NaN,0.0,0,1.0,"Data Scientist, Engineer, Software developer","Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,Intermediate,Yes,"comfortable submitting a PR, comfortable setti...","Python, C+ / C++, Java",Advanced,I am not at all familiar with PyMC.,"I am familiar, I have looked at the codebase.","Looking to grow my skills as a developer, and ...",NaN,English,No
1,Rochester USA,North America,United States,USD,840,43.157285,-77.615214,7/7/2022 10:47:41,Rochester,New York,https://github.com/Ali-McCommon,USA,Woman,she/her,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,"Analyst, Student","Yes, I identify as an under-represented person.","Working session #1: July 9, 2022 (Saturday), ...",WIA,No,No experience,No,no experience with Git,"Python, R, SQL",Beginner,I am not at all familiar with PyMC.,I am not at all familiar with this library.,Open source data is very valuable and I'd like...,Not sure,"English, ASL",No
2,New York USA,North America,United States,USD,840,40.712728,-74.006015,6/13/2022 19:56:59,New York,New York,https://github.com/mariyayb,USA,Woman,she/her,1.0,1,NaN,NaN,NaN,2.0,1,1.0,Data Scientist,I prefer not to answer this question.,"Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,No experience,No,comfortable setting up virtual environment,Python,Intermediate,I am not at all familiar with PyMC.,I am not at all familiar with this library.,gain experience and learn,Yes,English,No
3,New York USA,North America,United States,USD,840,40.712728,-74.006015,6/20/2022 20:25:26,New York,New York,https://github.com/zbraiterman,USA,Woman,"she/her, they/them",1.0,NaN,1.0,NaN,NaN,2.0,1,1.0,"Data Scientist, Analyst, Researcher","Yes, I identify as an under-represented person...","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,Yes,Intermediate,Yes,"comfortable submitting a PR, comfortable setti...","Python, R, MATLAB, C+ / C++, Java",Intermediate,I have used some PyMC for machine learning ana...,I have never looked at the source code.,To collaborate in building upon the data scien...,Yes,English,No
4,New York USA,North America,United States,USD,840,40.712728,-74.006015,6/22/2022 18:03:11,New York,New York,https://github.com/callistusndemo,USA,Man,he/him,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,Data Scientist,"Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Twitter,No,Beginner,Yes,"comfortable submitting a PR, comfortable setti...",Python,Beginner,I am not at all familiar with PyMC.,I am not at all familiar with this library.,I'd like to contribute more in the open source...,Yes,English,No
5,New York USA,North America,United States,USD,840,40.712728,-74.006015,6/23/2022 9:25:35,New York,New York,https://github.com/yyyyuans,USA,Woman,she/her,NaN,1,NaN,NaN,NaN,1.0,1,1.0,Engineer,"Yes, I identify as an under-represented person.","Pre-event office hours: July 2, 2022 (Saturda...",Meetup,No,No experience,No,"comfortable submitting a PR, comfortable cloni...","Python, Java",Intermediate,I am not at all familiar with PyMC.,"I am familiar, I have looked at the codebase.",I am interested in getting started in open sou...,Yes,English,No
6,New York USA,North America,United States,USD,840,40.712728,-74.006015,6/28/2022 12:35:09,New York,New York,https://github.com/Raharijao,USA,Man,he/him,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,Student,"Yes, I identify as an under-represented perso

In [28]:
df_locinfo2 = df_locinfo2.rename(columns={'province_state': 'state-province',
                        'Timestamp': 'timestamp',
                        'Attended' : 'attendance_status',
                       })

In [29]:
# Write out the dataframe to a file

df_locinfo2.to_csv('../data/data_derived/derived_geocodes_pymc.csv', index=False)

In [30]:
# Read csv file into a dataframe

#df_location = pd.read_csv('../data/derived_geocodes.csv', index_col=False)  

In [31]:
#df_location.head(3)

In [32]:
current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-08-23


In [33]:
print x

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(x)? (3945978432.py, line 1)

---

In [ ]:

#get_code("Bijuesca, Spain", 0)
#get_code("Dublin Ireland", 0)

#pprint(loc_codes)

In [ ]:
#type(loc_codes)

In [ ]:
# from pandas import DataFrame

# df_loc = DataFrame(loc_codes,columns=['place','continent','country','state','city','lat','lng'])

In [ ]:
df_loc=df_use.copy()

# delete some records temporarily

index_names = df_loc[df_loc['location'] == 'Harare Zimbabwe'].index 
df_loc.drop(index_names, inplace = True) 

index_names = df_loc[df_loc['location'] == 'Remote Global'].index 
df_loc.drop(index_names, inplace = True)

index_names = df_loc[df_loc['location'] == 'Guayaquil Ecuador'].index 
df_loc.drop(index_names, inplace = True)

index_names = df_loc[df_loc['location'] == 'Quito Ecuador'].index 
df_loc.drop(index_names, inplace = True)


index_names = df_loc[df_loc['location'] == 'Mérida Venezuela'].index 
df_loc.drop(index_names, inplace = True)

In [ ]:
count=-1
loc_list = list()

for i, row in df_loc.iterrows():
    max_rows = 300
    max_rows = 10
    if count <= max_rows:
        count+=1
        #print(count)
        #print(f"Index: {i}", f"{row['location']}", f"{row['location2']}")
        temp = row['location']
        #print(temp)
        temp2 = get_code(temp, 0)
        #print(temp2)
        loc_list.append(temp2)
        #print(f"{row['city']}")
        
    if count % 30 ==0:
        print(count)


---